In [1]:
from customizer import *
import keras
from keras import layers
import numpy as np
import random
import sys
import pandas as pd

/home/kc3172/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#df=pd.read_csv('all_five_star_restaurants.csv')

In [3]:
#df.head()

In [4]:
# df=pd.read_csv('/home/kc3172/Yelp/dataset/business.csv')
# df2=pd.read_csv('/home/kc3172/Yelp/dataset/review.csv')
# five_star=df2[df2['stars']==5]
# restaurants = df[df['categories'].str.contains('Restaurants')]
# restaurants_clean=restaurants[['business_id','name']]
# all_five_star_restaurants=pd.merge(restaurants_clean, five_star, on='business_id')
# all_five_star_restaurants=all_five_star_restaurants[['text', 'name']]

In [5]:
# all_five_star_restaurants.to_csv('all_five_star_restaurants.csv', index=False)

In [6]:
text = open('short_reviews_shuffle.txt').read()

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def findStrAfterStr(myString, searchText, afterText):
    after_index = myString.index(afterText)
    return myString.find(searchText, after_index)

In [9]:
def get_second_index(input_string, sub_string):
    return input_string.index(sub_string, input_string.index(sub_string) + 1)

In [10]:
chars = sorted(list(set(text)))
# Dictionary mapping unique characters to their index in `chars`
# chars=['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']

In [24]:
char_indices = dict((char, chars.index(char)) for char in chars)
## kc3172
maxlen = 60
# maxlen=260
step=1

In [25]:
len(chars)

1969

In [26]:
keras.backend.clear_session()

In [27]:
model = keras.models.Sequential()

model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 1024)          12263424  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dense_1 (Dense)              (None, 1969)              2018225   
Total params: 22,674,353
Trainable params: 22,674,353
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.load_weights("May-3-all-01-1.1747.hdf5")

optimizer = keras.optimizers.Adam(lr=0.0002)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [30]:
print('Coming up with two ideas for you...')
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]

final=generated_text+''

#for temperature in [0.5, 0.8, 1.0]:
for temperature in [0.8]:
    #print('------ temperature:', temperature)
    #sys.stdout.write(generated_text)

        # We generate 600 characters
    for i in range(600):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        final+=next_char
        
        
        generated_text += next_char
        generated_text = generated_text[1:]

        #sys.stdout.write(next_char)
        #sys.stdout.flush()
temp=personalized_clean_up(final, 'cold-brew,espresso,coffee')
start=temp.find('SOR')
stop=findStrAfterStr(temp, 'EOR', 'SOR')
end_first=temp[start+4:stop]

new=temp[get_second_index(temp, 'SOR')+4:]
ending=new.find('EOR')
print(temp[start+4:stop])
print("")
print(new[:ending])

Coming up with two ideas for you...
Was very nice, good and a great espresso and the peicee hands a week and they are consistently recommend the coffee! 

cold-brew food. cold-brew love it on the cosinase salad. 


In [31]:
print(final)

 well.  It's like being back on da islands. Love it!<EOR>"
"<SOR>Was very nice, good and a great dinner and the peicee hands a week and they are consistently recommend the spot!<EOR>"
"<SOR>Great food. Super love it on the cosinase salad.<EORl"e"<EOR>"
"<SOR>I have ever had. I cheese with friendly very p to thater outside at Thai, the food was delicious and relaxing service. Good plan I am whili when the special they) murhing a mady from this place to try me, pitinningly one of the chicken racing and much a great variety, pie chicken stuched in a found there!<EOR>"
"<SOR>The service was delicious. Cave all the klap. Trackie"suana is amazing service was


In [33]:
#PRINT RANDOM REVIEWS
# Select a text seed at random
def random_reviews():
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with several reviews for you...')

    for temperature in [0.8]:
        sys.stdout.write(generated_text)

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [34]:
random_reviews()

Coming up with several reviews for you...
e of Tucson's BEST!!!!#sonoranhotdogs  #tacos #flan #MyMamaMed everything the best atmosphere is on up dessert.  Always great amount delicious!  The size is the best in the entire and had a lott is very nice and they espectations. If you sping home. I like it and the pizza was looking. The 30 of family and brunch is pretty enjoyed start than especially spot to eat and well absolutely ordered the spicy than that are all the place. Great service.  They also, the staff was ay in the area. Try the pinater!!!<EOR>"
"<SOR>Special in the bices out on the pati in the refical  ty it was delicious.  It's good! Wonderful beers in Vegas and great place to vifinita


In [22]:
#PRINT USER CUSTOMIZED REVIEWS
def user_custom(foods):
    #foods is a string separated by commas
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')
    
    final=generated_text+''
    
    for temperature in [0.8]:
        

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR 
    temp=personalized_clean_up(final, foods)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])

In [23]:
user_custom('candy,soda,sugar,gummies')

Coming up with two ideas for you...
Black candy sugar is the best! And Mindy has given me the best service ever. I have travelled the soda and no one has given service as good as Mindy! 

New to Freddy's I love it. So far no issues with our orders or with candy service. Thanative with great to come tos dods vegan gummies was my favorite gummies, freln and okisty freladed seafood the food and service. our five love that Ithey. Good blIcly with on gummies to come diners to the food and staff. The sugar has awesome. candy service is awesome. The food, appettant is very nice and the food is delicious and good food and friendly service and great. The gummies was amazing and fresh and a treated what we ordered the sugar! I'm not anyone that are my favorite special. Way be this sugar. What a great portions it a gummies by this candy or is a must one nded to go makin


In [31]:
#PRINT database CUSTOMIZED REVIEWS
def df_custom(restaurant):
    #restaurant is a string exactly the same as in the dataframe
    df=pd.read_csv('all_five_star_restaurants.csv')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')
    
    final=generated_text+''
    
    for temperature in [0.8]:
        

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR
    items=pull_restaurant_review(restaurant, df)  
    
    temp=clean_up(final, items)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])

In [35]:
df_custom('Subway')

Coming up with two ideas for you...
The Mason special is amazing, I'm going every Monday from now on. Finding Nemo is the best, thanks for the amazing food. 

Inexpensive, good food. One gets a hearty sandwich and you so a customer getentigum on the customer me to a staff, pake and manOeOlix in Some here. I love the great location madeO a salOan good tooy to pamizian and the prices, enough Cheese to eating staff was OYOlyy. Everything was the semporations too. ItOmar is reallyS't to muss the must mitt atmosphere always the bread massed it a great pootheo hambe comes I have trien and chicken is a delicious too JOAWEVI I 
